##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introdução às variáveis

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/variable"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/variable.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/variable.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/variable.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Uma **variável** do TensorFlow é a maneira recomendável de representar o estado persistente compartilhado que seu programa manipula. Este guia aborda como criar, atualizar e gerenciar instâncias de `tf.Variable` no TensorFlow.

As variáveis são criadas e rastreadas pela classe `tf.Variable`. Uma `tf.Variable` representa um tensor com um valor que pode ser alterado ao executar ops nele. Ops específicas permitem que você leia e modifique os valores desse tensor. Bibliotecas de nível superior, como `tf.keras`, usam `tf.Variable` para armazenar parâmetros do modelo. 

## Configuração

Este notebook discute o posicionamento de variáveis.  Se você quiser ver em qual dispositivo estão suas variáveis, descomente esta linha.

In [ ]:
import tensorflow as tf

# Uncomment to see where your variables get placed (see below)
# tf.debugging.set_log_device_placement(True)

## Criação de uma variável

Para criar uma variável, forneça um valor inicial. A `tf.Variable` terá o mesmo `dtype` que o valor de inicialização.

In [ ]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

# Variables can be all kinds of types, just like tensors
bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

Uma variável se parece e age como um tensor, mas, na verdade, é uma estrutura de dados apoiada por um `tf.Tensor`. Como tensores, ela tem um `dtype` e um formato, além de poder ser exportada para o NumPy.

In [ ]:
print("Shape: ", my_variable.shape)
print("DType: ", my_variable.dtype)
print("As NumPy: ", my_variable.numpy())

A maioria das operações de tensores funcionam com as variáveis como esperado. No entanto, as variáveis não podem ter o formato alterado.

In [ ]:
print("A variable:", my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.math.argmax(my_variable))

# This creates a new tensor; it does not reshape the variable.
print("\nCopying and reshaping: ", tf.reshape(my_variable, [1,4]))

Conforme observado acima, as variáveis são apoiadas por tensores. Você pode reatribuir o tensor usando `tf.Variable.assign`. Chamar `assign` (geralmente) não aloca um novo tensor. Em vez disso, a memória do tensor existente é reutilizada.

In [ ]:
a = tf.Variable([2.0, 3.0])
# This will keep the same dtype, float32
a.assign([1, 2]) 
# Not allowed as it resizes the variable: 
try:
  a.assign([1.0, 2.0, 3.0])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

Se você usa uma variável como um tensor nas operações, geralmente operará no tensor de apoio.

A criação de novas variáveis a partir de outras existentes duplica os tensores de apoio. Duas variáveis não compartilham a mesma memória.

In [ ]:
a = tf.Variable([2.0, 3.0])
# Create b based on the value of a
b = tf.Variable(a)
a.assign([5, 6])

# a and b are different
print(a.numpy())
print(b.numpy())

# There are other versions of assign
print(a.assign_add([2,3]).numpy())  # [7. 9.]
print(a.assign_sub([7,9]).numpy())  # [0. 0.]

## Ciclos de vida, nomes e monitoramento

No TensorFlow baseado em Python, a instância `tf.Variable` tem o mesmo ciclo de vida que outros objetos no Python. Quando não há referências a uma variável, ela é desalocada automaticamente.

As variáveis também podem ser nomeadas, o que pode ajudar você com o rastreamento e a depuração delas. É possível dar o mesmo nome a duas variáveis.

In [ ]:
# Create a and b; they will have the same name but will be backed by
# different tensors.
a = tf.Variable(my_tensor, name="Mark")
# A new variable with the same name, but different value
# Note that the scalar add is broadcast
b = tf.Variable(my_tensor + 1, name="Mark")

# These are elementwise-unequal, despite having the same name
print(a == b)

Os nomes das variáveis são preservados ao salvar e carregar modelos. Por padrão, as variáveis nos modelos adquirem nomes exclusivos automaticamente, então você não precisa atribui-los por contra própria a menos que queira.

As variáveis são importantes para a diferenciação, mas algumas não precisam ser diferenciadas. Você pode desativar os gradientes de uma variável ao definir `trainable` como falso durante a criação. Um exemplo de variável que não precisa de gradientes é um contador de passos de treinamento.

In [ ]:
step_counter = tf.Variable(1, trainable=False)

## Posicionamento de variáveis e tensores

Para o melhor desempenho, o TensorFlow tentará colocar os tensores e as variáveis no dispositivo mais rápido compatível com o `dtype`. Isso significa que a maioria das variáveis serão colocadas em uma GPU, se disponível.

No entanto, você pode modificar isso. Neste snippet, coloque um tensor float e uma variável na CPU, mesmo se uma GPU estiver disponível. Ao ativar o registro de posicionamento de dispositivo (confira a [Configuração](#scrollTo=xZoJJ4vdvTrD)), você pode ver onde a variável foi colocada.

Observação: embora o posicionamento manual funcione, o uso das [estratégias de distribuição](distributed_training.ipynb) pode ser uma maneira mais conveniente e escalável de otimizar sua computação.

Ao executar este notebook em diferentes backends com e sem uma GPU, você verá registros diferentes. *Observe que o registro de posicionamento de dispositivo precisa ser ativado no início da sessão.*

In [ ]:
with tf.device('CPU:0'):

  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)

print(c)

É possível definir o local de uma variável ou um tensor em um dispositivo e fazer a computação em outro dispositivo. Isso introduzirá atraso, já que os dados precisam ser copiados entre dispositivos.

Você pode fazer isso, porém, se tiver vários workers de GPU, mas só quiser uma cópia das variáveis.

In [ ]:
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  # Element-wise multiply
  k = a * b

print(k)

Observação: como `tf.config.set_soft_device_placement` é ativado por padrão, mesmo se você executar esse código em um dispositivo sem GPU, ele ainda será executado. A etapa de multiplicação ainda ocorrerá na CPU.

Para saber mais sobre o treinamento distribuído, consulte o [guia](distributed_training.ipynb).

## Próximos passos

Para entender como as variáveis são geralmente usadas, veja nosso guia sobre a [diferenciação automática](autodiff.ipynb).